# Подготовка

Подготовим наши данные

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("X_train.csv")
dataset.describe(include="all")

,sku,categoryLevel1Id,categoryLevel2Id,brandId,property,userName,reting,date,comment,commentNegative,commentPositive
count,1.558700e+04,15587.000000,1.558700e+04,15587.000000,15587,15587,15587.000000,15587,15587,924,923
unique,NaN,NaN,NaN,NaN,2502,3232,NaN,2310,15513,694,922
top,NaN,NaN,NaN,NaN,"[{9079: '3495b78aaea0003e1e551bcf4da18861'}, {...",3d801da09e7d82668e226799d9db91dc,NaN,2011-06-03,нет,нет,Не рекомендую
freq,NaN,NaN,NaN,NaN,126,630,NaN,94,17,82,2
mean,2.470460e+07,341.629435,3.427723e+06,444.753577,NaN,NaN,4.118496,NaN,NaN,NaN,NaN
std,1.028728e+07,119.478142,1.186588e+06,494.612637,NaN,NaN,1.316966,NaN,NaN,NaN,NaN
min,1.000027e+06,101.000000,1.010102e+06,1.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
25%,2.000459e+07,207.000000,2.070202e+06,48.000000,NaN,NaN,4.000000,NaN,NaN,NaN,NaN
50%,2.002190e+07,405.000000,4.050206e+06,93.000000,NaN,NaN,5.000000,NaN,NaN,NaN,NaN
75%,3.001108e+07,411.000000,4.110102e+06,787.000000,NaN,NaN,5.000000,NaN,NaN,NaN,NaN


Воспользуемся полем comment, т.к. оно содержит текст отзыва и всегда заполнено. Тем не менее - заполним commentPositive/commentNegative.

In [3]:
dataset["commentPositive"] = dataset["commentPositive"].fillna("NA")
dataset["commentNegative"] = dataset["commentNegative"].fillna("NA")
dataset["rating"] = dataset["reting"].apply(round)

Разобьём отзывы на train / test (
    train будет использован при обучении изранной модели,
    а его подмножества будут использованы при кроссвалидации
).

In [4]:
from sklearn.model_selection import train_test_split

SEED = 42

comment = dataset["comment"]
rating = dataset["rating"]

comment_train, comment_test, rating_train, rating_test = train_test_split(np.array(dataset["comment"]),
                                                                          np.array(dataset["rating"]),
                                                                          random_state=SEED)

# Кроссвалидация

Определим функцию кроссвалидации по метрике MSE в 2 параллельных процесса

In [16]:
from sklearn.model_selection import cross_val_score

def cv(model, X, y):
    return -cross_val_score(model, X, y, scoring="neg_mean_squared_error", n_jobs=2)

# Константный классификатор

In [17]:
from sklearn.dummy import DummyClassifier

In [18]:
best_i = None
best_score = np.array([100])
for i in [1,2,3,4,5]:
    scores = cv(DummyClassifier("constant", constant=i), rating_train.reshape(-1, 1), rating_train)
    if scores.mean() < best_score.mean():
        best_i = i
        best_score = scores
print("Best constant : ", i)
print("Best constant scores : ", best_score)

Best constant :  5
Best constant scores :  [ 1.74095922  1.73973306  1.73915276]


# Линейные модели поверх CountVectorizer-а

Кроссвалидируем ряд моделей, в которых :
- на первой стадии - подсчитывается число вхождений различных строк
- на второй - рассчитывается значение линейной регрессии от преобразованных строк

При этом к регрессиям будут примены регуляризаторы:
- к первой - никакого (LinearRegression)
- к второй - L1 (Lasso)
- к третьей - L2 (Ridge)

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [20]:
cv(
    Pipeline([
        ("vectorizer", CountVectorizer()),
        ("regression", LinearRegression()),
    ]),
    comment_train,
    rating_train
)

array([ 44.71740315,  31.07388559,  61.04431389])

In [21]:
cv(
    Pipeline([
        ("vectorizer", CountVectorizer()),
        ("regression", Lasso()),
    ]),
    comment_train,
    rating_train
)

array([ 1.74792872,  1.7364251 ,  1.69093337])

In [22]:
cv(
    Pipeline([
        ("vectorizer", CountVectorizer()),
        ("regression", Ridge()),
    ]),
    comment_train,
    rating_train
)

array([ 1.38767043,  1.32933191,  1.28013101])

Видим, что лучший результат возвращает Ridge (L2), линейная же модель переобучается.

# Lasso/Ridge поверх Tf-Idf

Кроссвалидируем регрессии с L1/L2 регуляризаторами, применяемые к результату tf-idf преобразования

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
cv(
    Pipeline([
        ("vectorizer", TfidfVectorizer()),
        ("regression", Lasso()),
    ]),
    comment_train,
    rating_train
)

array([ 1.74792872,  1.7364251 ,  1.69093337])

In [25]:
cv(
    Pipeline([
        ("vectorizer", TfidfVectorizer()),
        ("regression", Ridge()),
    ]),
    comment_train,
    rating_train
)

array([ 0.9803364 ,  0.98791351,  0.94199887])

Видим, что результат Ridge ощутимо лучше, чем Lasso

# Стемминг

Попробуем снизить размерность пространства признаков, применив перед tf-idf стемминг (
    который оставит от слова только "основную" часть - таким образом, одному слову будет соответствовать более 1 признака)

In [27]:
from stem_transformer import stem_transformer

In [28]:
cv(
    Pipeline([
        ("stem", stem_transformer()),
        ("vectorizer", TfidfVectorizer()),
        ("regression", Ridge()),
    ]),
    comment_train,
    rating_train
)

array([ 0.96504935,  0.96954719,  0.94524739])

## Конкатенация с токеном отрицания

В некоторых ситуациях нам нужно обрабатывать не слова (после стемминга), а пары слов.
Например - "не пожалеете" (очевиден положительный окрас) стоит считать одним токеном.
Т.к. в противном случае - мы получаем 2 токена: ["не", "пожалеете"], из которых 1 не имеет выраженного окраса, 
    другой же - имеет отрицательный. Очевидно, стоит обработать такие пары токены.
Применённый здесь алгоритм таков:
    * разбиваем текст после стемминга на отдельные токены
    * для каждой пары из токена и следующего токена:
        * если токен - "не" - склеиваем его со следующим токеном, добавляем к результирующему списку, пропускаем следующую пару
        * иначе - добавляем к результирующему списку
    * склеиваем результирующий список
Тогда пара токенов "не", "пожалеете" будет представлена 1 токеном "не_пожалеете"

In [30]:
from inverse_token_concatenation import inverse_token_concatenation

In [31]:
cv(
    Pipeline([
        ("stem", stem_transformer()),
        ("invcon", inverse_token_concatenation()),
        ("vectorizer", TfidfVectorizer()),
        ("regression", Ridge()),
    ]),
    comment_train,
    rating_train
)

array([ 0.8845893 ,  0.89362136,  0.86758655])

Видим существенное улучшение результата

# Ограничение выходного значения

В нашем случае выходные значения находятся в диапазоне [1;5].
Однако в общем случае Ridge-регрессия способна выдать значения вне этого диапазона, если не наложить на неё ограничений. Проверим работу модели, в которой применяется такое ограничение:

* X - предсказание Ridge
* X[X < 1] = 1 - заменяем 1 все значения, меньшие, чем 1
* X[X > 5] = 5 - заменяем 5 все значения, большие, чем 5

In [32]:
from misc import TransformRidge
from output_range_transformation import output_range_transformation

In [33]:
cv(
    Pipeline([
        ("stem", stem_transformer()),
        ("invcon", inverse_token_concatenation()),
        ("vectorizer", TfidfVectorizer()),
        ("regression", TransformRidge()),
        ("output", output_range_transformation(1, 5)),
    ]),
    comment_train,
    rating_train
)

array([ 0.85393971,  0.86415039,  0.83434697])

Видим, что значение ошибки и в самом деле упало.

# Оценка полярности комментария

С помощью polyglot можно оценить полярность текста следующим образом:
* токенизовать его
* создать пустой список полярностей токенов (1 - положительная, 0 - нейтральная, -1 - отрицательная)
* для каждой пары токена и следующего за ним токена:
    * если токен="не"
        * добавить в список обращённую полярность следующего токена
    * иначе
        * дабавить в список полярность текущего токена
* positive - количество токенов с положительной полярностью
* negative - с негативной
* vec = (positive, negative) - ненормализованный двумерный вектор полярностей
* если длинна vec = 0:
    * вернуть (0, 0, 1)
* иначе:
    * vecNorm = (vec / length(vec)) ^ 2 - нормализуем вектор и возводим элементы в квадрат
    * вернуть (vecNorm(0), vecNorm(1), 1 - sum(vecNorm))
В итоге компоненты вектора связаны с :
- количеством слов с положительным окрасом (должен быть высок у текстов с положительным окрасом)
- количеством слов с отрицательным окрасом (должен быть высок у текстов с отрицательным окрасом)
- разницей между 1 и остальными компонентами (должен быть высок у текстов без выраженного окраса)

In [34]:
from polarity_vectorizer import polarity_vectorizer
from sklearn.pipeline import FeatureUnion

In [35]:
cv(
    Pipeline([
        ("features", FeatureUnion([
            ("stem_tfidf", Pipeline([
                ("stem", stem_transformer()),
                ("invcon", inverse_token_concatenation()),
                ("vectorizer", TfidfVectorizer()),
            ])),
            ("polarity", polarity_vectorizer("ru")),
        ])),
        ("regression", TransformRidge()),
        ("output", output_range_transformation(1, 5)),
    ]),
    comment_train,
    rating_train
)

array([ 0.85013574,  0.86464798,  0.83594452])

Видим небольшое улучшение результата.

# Сохранение готовой модели

Построим итоговую модель, обучим её на всем обучающем подмножестве

In [36]:
model = Pipeline([
    ("features", FeatureUnion([
        ("stem_tfidf", Pipeline([
            ("stem", stem_transformer()),
            ("invcon", inverse_token_concatenation()),
            ("vectorizer", TfidfVectorizer()),
        ])),
        ("polarity", polarity_vectorizer("ru")),
    ])),
    ("regression", TransformRidge()),
    ("output", output_range_transformation(1, 5)),
])

In [37]:
model.fit(comment_train, rating_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('stem_tfidf', Pipeline(memory=None,
     steps=[('stem', FunctionTransformer(accept_sparse=False,
          func=<function _stem at 0x000000CFBAC731E0>, inv_kw_args=None,
          inverse_func=None,
          kw_args={'stemmer': <...0>,
              kw_args={'min': 1, 'max': 5}, pass_y='deprecated',
              validate=False))])

Посчитаем значение MSE на тестовой выборке

In [42]:
from sklearn.metrics import mean_squared_error

mean_squared_error(rating_test, model.predict(comment_test))

0.90137795312416091

Сохраним модель

In [41]:
import pickle

with open("model.pkl", "wb") as target:
    pickle.dump(model, target)